In [1]:
import sys
sys.path.append('../')

from data_partitioner import *
from parallel_tagging import *
from biberplus.tagger import load_config, load_pipeline, calculate_tag_frequencies
from biberplus.reducer import encode_text
from biberplus.tagger.tagger_utils import load_tokenizer
# import gzip

INFO:numexpr.utils:Note: detected 160 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 160 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
author_subreddit_counts = {}

In [3]:
# with gzip.open('/shared/4/datasets/long-reddit/RC_2022-01.gz', 'rb') as f:
#     # Decompress and decode the file
#     decompressed_data = f.read().decode('utf-8')
    
#     # Split the data into individual samples based on newline delimiter
#     samples = decompressed_data.split('\n')
    
#     # Iterate over individual samples
#     for sample in samples:
#         # Process each sample here
#         print(sample)
#         break

## Partition the file into 100 parts

In [4]:
input_file = '/shared/4/datasets/long-reddit/RC_2022-02.gz'
output_directory = '/shared/3/projects/hiatus/tagged_data/partitions/reddit'

In [ ]:
partition_file(input_file, output_directory, chunks=100)

INFO:root:1685373 lines in file
INFO:root:Chunk size: 16854 lines
INFO:root:Saving /shared/3/projects/hiatus/tagged_data/partitions/reddit/partition-1.jsonl.gz
INFO:root:Saving /shared/3/projects/hiatus/tagged_data/partitions/reddit/partition-2.jsonl.gz
INFO:root:Saving /shared/3/projects/hiatus/tagged_data/partitions/reddit/partition-3.jsonl.gz
INFO:root:Saving /shared/3/projects/hiatus/tagged_data/partitions/reddit/partition-4.jsonl.gz
INFO:root:Saving /shared/3/projects/hiatus/tagged_data/partitions/reddit/partition-5.jsonl.gz
INFO:root:Saving /shared/3/projects/hiatus/tagged_data/partitions/reddit/partition-6.jsonl.gz


## Tag each partition with 1 CPU 

Set nice value low so we don't hog the server

**Tagger config**

In [ ]:
config = load_config()
config.update({'use_gpu': False, 
               'biber': True,
               'binary_tags': True, 
               'function_words': True,
               'token_normalization': 100})
tokenizer = load_tokenizer(use_gpu=False)
config

In [ ]:
%%time
import warnings

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

tag_partitions(config,
               input_directory='/shared/3/projects/hiatus/tagged_data/partitions/reddit/',
               output_directory='/shared/3/projects/hiatus/tagged_data/long-reddit/',
               num_workers=50,
               post_counts=author_subreddit_counts,
               default_niceness=-1
               )

In [ ]:
%%time

# Write the post counts to a TSV file
write_counts_to_tsv(author_subreddit_counts, "/shared/3/projects/hiatus/tagged_data/long-reddit/author_counts.tsv")

## Join the partitioned files

In [ ]:
join_tagged_files(input_directory='/shared/2/projects/hiatus/partitions/reddit/',
                  ouput_file='/shared/3/projects/hiatus/reddit/corpus_tagged.jsonl')

## Delete the partitions directory

In [ ]:
delete_partitioned_files('/shared/2/projects/hiatus/partitions/reddit/')

In [ ]:
delete_partitioned_files('/shared/3/projects/hiatus/partitions/reddit/')